In [ ]:
#%cd ~ && rm -rf NAS_LLM
!git clone -b features https://github.com/WpythonW/NAS_LLM.git
!uv pip install --system "numpy<2" optuna sympy
%cd NAS_LLM/Informer2020

Cloning into 'NAS_LLM'...
remote: Enumerating objects: 280, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 280 (delta 38), reused 70 (delta 18), pack-reused 175 (from 1)
Receiving objects: 100% (280/280), 3.31 MiB | 26.86 MiB/s, done.
Resolving deltas: 100% (122/122), done.
Using Python 3.12.12 environment at: /usr
Resolved 14 packages in 142ms
Prepared 3 packages in 297ms
Uninstalled 1 package in 30ms
Installed 3 packages in 16ms
 + colorlog==6.10.1
 - numpy==2.0.2
 + numpy==1.26.4
 + optuna==4.6.0
/content/NAS_LLM/Informer2020


In [ ]:
%cd NAS_LLM/Informer2020
from utils.tools import dotdict
from exp.exp_informer import Exp_Informer
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.dates import DateFormatter
from matplotlib.ticker import MaxNLocator
import os, optuna
import torch
import numpy as np
from optuna.storages import JournalStorage
from optuna.storages.journal import JournalFileBackend
import sys
import json
from pathlib import Path
from datetime import datetime

/content/NAS_LLM/Informer2020


In [3]:
!rm "/content/NAS_LLM/Informer2020/optunaETTh2.log"
!rm "/content/NAS_LLM/Informer2020/optunaETTh2.log.lock"
!rm "/content/NAS_LLM/Informer2020/ETTh1;pred_len=168;OPTUNA.json"

rm: cannot remove '/content/NAS_LLM/Informer2020/optunaETTh2.log.lock': No such file or directory


In [4]:
class TrialLogger:
    def __init__(self, filepath='trials.json'):
        self.filepath = Path(filepath)
        self.trials = self._load()

    def _load(self):
        return json.loads(self.filepath.read_text()) if self.filepath.exists() else []

    def _save(self):
        self.filepath.write_text(json.dumps(self.trials, indent=2))

    def add(self, trial_number, params, measures):
        self.trials.append({
            'trial': trial_number,
            'timestamp': datetime.now().isoformat(),
            **{k: float(v) for k, v in measures.items()},
            'seq_len': params['seq_len'],
            'label_len': params['label_len'],
            'e_layers': params['e_layers'],
            'n_heads': params['n_heads'],
            'factor': params['factor']
        })
        self._save()

def objective(trial):
    args = dotdict()

    # Фиксированные параметры
    args.model = 'informer'
    args.data = 'custom'
    args.root_path = './'
    args.data_path = f'../data/{dataset_name}_prepared.csv'
    args.features = 'MS'
    args.target = 'OT'
    args.freq = 'h'
    args.checkpoints = './informer_checkpoints'
    args.d_model = 512
    args.d_layers = 2
    args.d_ff = 2048
    args.dropout = 0.05
    args.attn = 'prob'
    args.embed = 'timeF'
    args.activation = 'gelu'
    args.distil = True
    args.output_attention = False
    args.mix = True
    args.padding = 0
    args.batch_size = 32
    args.learning_rate = 0.0001
    args.loss = 'mse'
    args.lradj = 'type1'
    args.train_epochs = 6
    args.patience = 3
    args.num_workers = 0
    args.itr = 1
    args.des = 'optuna'
    args.use_amp = False

    # Используем обе GPU
    args.use_gpu = True
    args.use_multi_gpu = True
    args.devices = '0'
    args.device_ids = [0]
    args.gpu = 0

    # Перебираемые параметры с constraint
    args.seq_len = trial.suggest_categorical('seq_len', [96, 168, 336, 480, 720])
    args.label_len = trial.suggest_categorical('label_len', [24, 48, 96])

    # label_len должен быть <= seq_len
    if args.label_len > args.seq_len:
        raise optuna.TrialPruned()

    args.pred_len = pred_len ##########################CHANGE
    args.e_layers = trial.suggest_categorical('e_layers', [2, 3, 4, 6])
    args.n_heads = trial.suggest_categorical('n_heads', [8, 16])
    args.factor = trial.suggest_categorical('factor', [3, 5, 8, 10])

    args.enc_in = args.dec_in = 7
    args.c_out = 1
    args.detail_freq = args.freq
    args.freq = args.freq[-1:]

    exp = Exp_Informer(args)
    setting = f'informer_trial{trial.number}'

    exp.train(setting)
    mae_val, mse_val = exp.test(setting, flag='val')
    mae_test, mse_test = exp.test(setting, flag='test')

    measures = {'mae_val': mae_val, 'mse_val': mse_val, 'mae_test': mae_test, 'mse_test': mse_test}

    logger.add(trial.number, trial.params, measures)

    return mse_val

pred_len=168
dataset_name = 'ETTh1'
logger = TrialLogger(f'{dataset_name};pred_len={pred_len};OPTUNA.json')
storage = JournalStorage(JournalFileBackend('./optunaETTh2.log'))

study = optuna.create_study(direction='minimize',
                            storage=storage,
                            study_name=f'opt,pred_len={pred_len}',
                            load_if_exists=True)
study.optimize(objective, n_trials=35)

print(f'Best MSE: {study.best_value:.4f}')
print(f'Best params: {study.best_params}')

[I 2025-11-17 16:56:17,455] A new study created in Journal with name: opt,pred_len=168


Use GPU: cuda:0
train 11931
val 1575
test 3317
	iters: 100, epoch: 1 | loss: 0.2518760
	speed: 0.0998s/iter; left time: 212.9771s
	iters: 200, epoch: 1 | loss: 0.1412952
	speed: 0.1006s/iter; left time: 204.6128s
	iters: 300, epoch: 1 | loss: 0.1435901
	speed: 0.1004s/iter; left time: 193.9980s
Epoch: 1 cost time: 37.322211265563965
Epoch: 1, Steps: 372 | Train Loss: 0.2101685 Vali Loss: 0.9936406 Test Loss: 0.3600029
Validation loss decreased (inf --> 0.993641).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.1500351
	speed: 0.2386s/iter; left time: 420.1912s
	iters: 200, epoch: 2 | loss: 0.1376794
	speed: 0.0999s/iter; left time: 166.0015s
	iters: 300, epoch: 2 | loss: 0.1149949
	speed: 0.1000s/iter; left time: 156.0984s
Epoch: 2 cost time: 37.176764249801636
Epoch: 2, Steps: 372 | Train Loss: 0.1235618 Vali Loss: 0.9771312 Test Loss: 0.3381346
Validation loss decreased (0.993641 --> 0.977131).  Saving model ...
Updating learning rate to 5e-05
	iter

[I 2025-11-17 17:00:47,828] Trial 0 finished with value: 0.7541407346725464 and parameters: {'seq_len': 96, 'label_len': 24, 'e_layers': 4, 'n_heads': 16, 'factor': 8}. Best is trial 0 with value: 0.7541407346725464.


test shape: (103, 32, 168, 1) (103, 32, 168, 1)
test shape: (3296, 168, 1) (3296, 168, 1)
mse:0.29060307145118713, mae:0.43146342039108276
Use GPU: cuda:0
train 11859
val 1575
test 3317
	iters: 100, epoch: 1 | loss: 0.2909917
	speed: 0.1419s/iter; left time: 300.8889s
	iters: 200, epoch: 1 | loss: 0.1887119
	speed: 0.1425s/iter; left time: 288.0476s
	iters: 300, epoch: 1 | loss: 0.1252031
	speed: 0.1428s/iter; left time: 274.2948s
Epoch: 1 cost time: 52.78212523460388
Epoch: 1, Steps: 370 | Train Loss: 0.1938815 Vali Loss: 0.8619397 Test Loss: 0.3224031
Validation loss decreased (inf --> 0.861940).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.1100410
	speed: 0.3518s/iter; left time: 615.9484s
	iters: 200, epoch: 2 | loss: 0.0951190
	speed: 0.1442s/iter; left time: 238.0714s
	iters: 300, epoch: 2 | loss: 0.0841915
	speed: 0.1443s/iter; left time: 223.8089s
Epoch: 2 cost time: 53.36601138114929
Epoch: 2, Steps: 370 | Train Loss: 0.1067982 Vali Loss: 

[I 2025-11-17 17:05:14,410] Trial 1 finished with value: 0.8605110049247742 and parameters: {'seq_len': 168, 'label_len': 96, 'e_layers': 4, 'n_heads': 8, 'factor': 10}. Best is trial 0 with value: 0.7541407346725464.


...

test shape: (103, 32, 168, 1) (103, 32, 168, 1)
test shape: (3296, 168, 1) (3296, 168, 1)
mse:0.17518343031406403, mae:0.3394048810005188
Use GPU: cuda:0
train 11307
val 1575
test 3317
	iters: 100, epoch: 1 | loss: 0.2402520
	speed: 0.2836s/iter; left time: 572.5960s
	iters: 200, epoch: 1 | loss: 0.1620077
	speed: 0.2851s/iter; left time: 547.0819s
	iters: 300, epoch: 1 | loss: 0.1356747
	speed: 0.2853s/iter; left time: 518.9942s
Epoch: 1 cost time: 100.63065361976624
Epoch: 1, Steps: 353 | Train Loss: 0.1905633 Vali Loss: 0.3725385 Test Loss: 0.2619552
Validation loss decreased (inf --> 0.372539).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.1318642
	speed: 0.6457s/iter; left time: 1075.7223s
	iters: 200, epoch: 2 | loss: 0.1011929
	speed: 0.2851s/iter; left time: 446.3989s
	iters: 300, epoch: 2 | loss: 0.1147460
	speed: 0.2849s/iter; left time: 417.7157s
Epoch: 2 cost time: 100.60094594955444
Epoch: 2, Steps: 353 | Train Loss: 0.1146863 Vali Loss

[I 2025-11-17 21:29:05,175] Trial 33 finished with value: 0.31771716475486755 and parameters: {'seq_len': 720, 'label_len': 24, 'e_layers': 4, 'n_heads': 16, 'factor': 8}. Best is trial 27 with value: 0.1836838573217392.


test shape: (103, 32, 168, 1) (103, 32, 168, 1)
test shape: (3296, 168, 1) (3296, 168, 1)
mse:0.23577921092510223, mae:0.38330402970314026
Use GPU: cuda:0
train 11307
val 1575
test 3317
	iters: 100, epoch: 1 | loss: 0.1961417
	speed: 0.2589s/iter; left time: 522.7395s
	iters: 200, epoch: 1 | loss: 0.1322304
	speed: 0.2606s/iter; left time: 500.0676s
	iters: 300, epoch: 1 | loss: 0.1696085
	speed: 0.2602s/iter; left time: 473.3831s
Epoch: 1 cost time: 91.89830160140991
Epoch: 1, Steps: 353 | Train Loss: 0.1897408 Vali Loss: 0.5065490 Test Loss: 0.3695538
Validation loss decreased (inf --> 0.506549).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.1179397
	speed: 0.5986s/iter; left time: 997.2640s
	iters: 200, epoch: 2 | loss: 0.0922418
	speed: 0.2611s/iter; left time: 408.9190s
	iters: 300, epoch: 2 | loss: 0.0981456
	speed: 0.2609s/iter; left time: 382.5140s
Epoch: 2 cost time: 92.13038444519043
Epoch: 2, Steps: 353 | Train Loss: 0.1220156 Vali Loss: 

[I 2025-11-17 21:40:37,427] Trial 34 finished with value: 0.342081755399704 and parameters: {'seq_len': 720, 'label_len': 24, 'e_layers': 2, 'n_heads': 8, 'factor': 10}. Best is trial 27 with value: 0.1836838573217392.


test shape: (103, 32, 168, 1) (103, 32, 168, 1)
test shape: (3296, 168, 1) (3296, 168, 1)
mse:0.28760814666748047, mae:0.42922866344451904
Best MSE: 0.1837
Best params: {'seq_len': 720, 'label_len': 24, 'e_layers': 4, 'n_heads': 8, 'factor': 3}


In [ ]:
!rm "/kaggle/working/NAS_LLM/Informer2020/ETTh2;pred_len=168;OPTUNA.json"

In [5]:
!ls

 data				   informer_checkpoints   optunaETTh2.log
'ETTh1;pred_len=168;OPTUNA.json'   main_informer.py	  results
 exp				   models		  utils
